In [ ]:
import json, yaml

import scipy.stats as ss
import scipy.optimize as sopt

import numpy as np
import pandas as pd
import geopandas as gpd

import plotly.express as px

In [ ]:
if "snakemake" in locals():
    demand_path = snakemake.input["demand"]
    routes_path = snakemake.input["routes"]
    scenario_path = snakemake.input["scenario"]
    output_path = snakemake.output[0]

else:
    demand_path = "../../results/demand/merged_seed1000.gpkg"
    routes_path = "../../results/routing/routes_seed1000.json"
    scenario_path = "../../resources/mode_choice/scenario.yml"
    output_path = "../../results/mode_choice/choices.gpkg"

In [ ]:
# Load parameters
with open(scenario_path) as f:
    parameters = yaml.load(f, yaml.FullLoader)

model = parameters["model"]

In [ ]:
# Load routes
with open(routes_path) as f:
    routes = json.load(f)

In [ ]:
# Load demand
df_demand = gpd.read_file(demand_path)

In [ ]:
# Convert to data frames
df_road = pd.DataFrame.from_records(routes["road_router"])[[
    "request_index", "in_vehicle_time_min", "in_vehicle_distance_km"
]]

df_road.columns = ["road_{}".format(c) if c != "request_index" else c for c in df_road.columns]

df_transit = pd.DataFrame.from_records(routes["transit_router"])[[
    "request_index", "access_walk_time_min", "egress_walk_time_min", "transfer_walk_time_min",
    "initial_wait_time_min", "transfer_wait_time_min",
    "in_vehicle_travel_time"
]].rename(columns = { "in_vehicle_travel_time": "in_vehicle_travel_time_min" })

df_transit.columns = ["transit_{}".format(c) if c != "request_index" else c for c in df_transit.columns]

In [ ]:
# Merge everything
df_demand = pd.merge(df_demand, df_road, on = "request_index")
df_demand = pd.merge(df_demand, df_transit, on = "request_index")

In [ ]:
# Scale travel time
df_demand["road_in_vehicle_time_min"] *= parameters["congestion_factor"]

In [ ]:
# Attach VTTS parameters
for profile in df_demand["profile"].unique():
    value = parameters["model"]["vtts"][profile]
    f_profile = df_demand["profile"] == profile
    df_demand.loc[f_profile, "vtts"] = value

In [ ]:
# Construct utility for car (without ASC)
df_demand["car_utility"] = 0.0

car_travel_time_h = df_demand["road_in_vehicle_time_min"] / 60.0
df_demand["car_utility"] -= car_travel_time_h * df_demand["vtts"]

car_cost = df_demand["road_in_vehicle_distance_km"] * parameters["modes"]["car"]["cost_per_km"]
car_cost /= df_demand["group_size"]
df_demand["car_utility"] -= car_cost

In [ ]:
# Construct utility for taxi (without ASC)
df_demand["taxi_utility"] = 0.0

taxi_travel_time_h = df_demand["road_in_vehicle_time_min"] / 60.0
df_demand["taxi_utility"] -= taxi_travel_time_h * df_demand["vtts"]

taxi_cost = df_demand["road_in_vehicle_distance_km"] * parameters["modes"]["taxi"]["cost_per_km"]
taxi_cost += parameters["modes"]["taxi"]["cost_per_trip"]
taxi_cost /= df_demand["group_size"]
df_demand["taxi_utility"] -= taxi_cost

vwts = df_demand["vtts"] * model["wait_time_factor"] * parameters["modes"]["taxi"]["vwts_factor"]
taxi_wait_time_h = parameters["modes"]["taxi"]["wait_time_min"] / 60.0
df_demand["taxi_utility"] -= taxi_wait_time_h * vwts

In [ ]:
# Construct utility for CCAM (without ASC)
df_demand["ccam_utility"] = 0.0

ccam_travel_time_h = df_demand["road_in_vehicle_time_min"] / 60.0
df_demand["ccam_utility"] -= ccam_travel_time_h * df_demand["vtts"]

ccam_cost = df_demand["road_in_vehicle_distance_km"] * parameters["modes"]["ccam"]["cost_per_km"]
ccam_cost += parameters["modes"]["ccam"]["cost_per_trip"]
ccam_cost /= df_demand["group_size"]
df_demand["ccam_utility"] -= ccam_cost

vwts = df_demand["vtts"] * model["wait_time_factor"] * parameters["modes"]["ccam"]["vwts_factor"]
ccam_wait_time_h = parameters["modes"]["ccam"]["wait_time_min"] / 60.0
df_demand["ccam_utility"] -= ccam_wait_time_h * vwts

In [ ]:
# Construct utility for transit (without ASC)
df_demand["transit_utility"] = 0.0

transit_travel_time_h = df_demand["transit_in_vehicle_travel_time_min"] / 60.0
df_demand["transit_utility"] -= transit_travel_time_h * df_demand["vtts"]

transit_cost = parameters["modes"]["transit"]["cost_per_person"]
df_demand["transit_utility"] -= transit_cost

transit_wait_time_h = df_demand["transit_initial_wait_time_min"] / 60.0 + df_demand["transit_transfer_wait_time_min"] / 60.0
vwts = df_demand["vtts"] * model["wait_time_factor"]
df_demand["transit_utility"] -= transit_wait_time_h * vwts

transit_walk_time_h = df_demand["transit_access_walk_time_min"] / 60.0 + df_demand["transit_egress_walk_time_min"] / 60.0 + df_demand["transit_transfer_walk_time_min"] / 60.0
vwts = df_demand["vtts"] * model["walk_time_factor"]
df_demand["transit_utility"] -= transit_walk_time_h * vwts

In [ ]:
# Extract relevant information
df_demand = df_demand[[
    "request_index", "origin_x", "origin_y", "destination_x", "destination_y", "geometry",
    "profile", "group_size",
    "car_utility", "transit_utility", "taxi_utility", "ccam_utility"
]].copy()

In [ ]:
# Initialize RNG and Gumbel distribution
random_state = np.random.RandomState(seed = 0)
distribution = ss.gumbel_r()

In [ ]:
# Sample random preference terms
df_demand["car_utility"] += distribution.rvs(len(df_demand), random_state = random_state)
df_demand["transit_utility"] += distribution.rvs(len(df_demand), random_state = random_state)
df_demand["taxi_utility"] += distribution.rvs(len(df_demand), random_state = random_state)
df_demand["ccam_utility"] += distribution.rvs(len(df_demand), random_state = random_state)

In [ ]:
# Calibration of the model
modes = ["car", "transit", "taxi"]

target = np.array([parameters["calibration"][mode] for mode in modes])
target = target / np.sum(target)

# constants = { car, taxi }
def objective(constants):
    utilities = df_demand[["{}_utility".format(m) for m in modes]].values
    utilities[:, modes.index("car")] += constants[0]
    utilities[:, modes.index("taxi")] += constants[1]

    selection = np.argmax(utilities, axis = 1)
    shares = np.array([np.count_nonzero(selection == k) for k in range(3)])
    shares = shares / np.sum(shares)

    return np.sum(np.abs(shares - target))

result = sopt.dual_annealing(
    objective, bounds = [[-10.0, 10.0], [-10.0, 10.0]], 
    x0 = [0.0, 0.0], maxiter = 1000
)

assert result.success
assert result.fun < 1e-3

# Insert constants
df_demand["car_utility"] += result.x[0]
df_demand["taxi_utility"] += result.x[1]
df_demand["ccam_utility"] += result.x[1] # CCAM gets taxi offset

In [ ]:
# Perform choice based on calibrated utilities
utilities = df_demand[["{}_utility".format(m) for m in modes]].values    
selection = np.argmax(utilities, axis = 1)
df_demand["mode"] = [modes[k] for k in selection]

In [ ]:
df_plot = df_demand.groupby("mode").size().reset_index(name = "share")
df_plot["share"] /= df_plot["share"].sum()
df_plot["source"] = "model"
df_plot = pd.concat([df_plot, pd.DataFrame({ "mode": modes, "share": target, "source": "reference" })])
px.bar(df_plot, x = "mode", y = "share", color = "source", barmode = "group",
    title = "Model fit after constant calibration")

In [ ]:
# Remove columns
df_demand = df_demand.drop(columns = [
    "car_utility", "transit_utility", "taxi_utility", "ccam_utility"
])

In [ ]:
# Output
df_demand.to_file(output_path)